In [1]:
from utils import controlled_compute, logger, tqdmbar, show_dask_progress, load_zarr, ZARRLOC
from typing import Generator, IO, Any, Tuple, List, Union, Dict, Optional
from scipy.sparse import csr_matrix, coo_matrix
import nexusformat.nexus as nx
import pandas as pd
import numpy as np
import h5py
import sys
import os
import math
import zarr

In [2]:
# __init__.py at ./
sys.path.append('..')
import scarf

Scarf is not installed


In [3]:
%load_ext autotime

time: 82.9 μs (started: 2024-08-30 01:46:43 +05:30)


# Existing Datasets

In [ ]:
# This dataset is in Cellranger (10x) HDF5 format.
scarf.fetch_dataset(
    dataset_name='tenx_10K_pbmc-v1_atacseq',
    save_path='./test_data'
)

In [ ]:
# This dataset is in MTX format along with barcodes and features TSV files.
scarf.fetch_dataset(
    dataset_name='xin_1K_pancreas_rnaseq',
    save_path='./test_data'
)

# Reader

In [4]:
reader = scarf.CrDirReader('./test_data/xin_1K_pancreas_rnaseq', is_filtered=True)
writer = scarf.CrToZarr(reader, '../../tmp/zarr-converted/xin_1K_pancreas_rnaseq_dev.zarr')

time: 148 ms (started: 2024-08-30 01:46:47 +05:30)


In [6]:
writer.dump()

  0%|                                                                                                         …

time: 6.91 s (started: 2024-08-30 01:46:57 +05:30)


# Check

In [8]:
orginal_zarr = zarr.open('../../tmp/zarr-converted/xin_1K_pancreas_rnaseq_org.zarr')
orginal_zarr_count = orginal_zarr['RNA/counts']
new_zarr = zarr.open('../../tmp/zarr-converted/xin_1K_pancreas_rnaseq_dev.zarr')
new_zarr_count = new_zarr['RNA/counts']

time: 10.4 ms (started: 2024-08-30 01:49:40 +05:30)


In [9]:
print(orginal_zarr.info), print(orginal_zarr_count.info)

Name        : /
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 2
No. arrays  : 0
No. groups  : 2
Groups      : RNA, cellData

Name               : /RNA/counts
Type               : zarr.core.Array
Data type          : uint32
Shape              : (1600, 39846)
Chunk shape        : (1000, 1000)
Order              : C
Read-only          : False
Compressor         : Blosc(cname='lz4', clevel=5, shuffle=BITSHUFFLE,
                   : blocksize=0)
Store type         : zarr.storage.DirectoryStore
No. bytes          : 255014400 (243.2M)
No. bytes stored   : 96038826 (91.6M)
Storage ratio      : 2.7
Chunks initialized : 80/80



(None, None)

time: 6.02 ms (started: 2024-08-30 01:49:41 +05:30)


In [10]:
print(new_zarr.info), print(new_zarr_count.info)

Name        : /
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 2
No. arrays  : 0
No. groups  : 2
Groups      : RNA, cellData

Name               : /RNA/counts
Type               : zarr.core.Array
Data type          : uint32
Shape              : (1600, 39846)
Chunk shape        : (1000, 1000)
Order              : C
Read-only          : False
Compressor         : Blosc(cname='lz4', clevel=5, shuffle=BITSHUFFLE,
                   : blocksize=0)
Store type         : zarr.storage.DirectoryStore
No. bytes          : 255014400 (243.2M)
No. bytes stored   : 96038826 (91.6M)
Storage ratio      : 2.7
Chunks initialized : 80/80



(None, None)

time: 5.61 ms (started: 2024-08-30 01:49:41 +05:30)


In [11]:
assert orginal_zarr_count.nchunks == new_zarr_count.nchunks

time: 622 μs (started: 2024-08-30 01:49:42 +05:30)


In [12]:
assert orginal_zarr_count.shape == new_zarr_count.shape

time: 225 μs (started: 2024-08-30 01:49:42 +05:30)


In [13]:
assert orginal_zarr_count.chunks == new_zarr_count.chunks

time: 271 μs (started: 2024-08-30 01:49:42 +05:30)


In [14]:
chunks = math.ceil(orginal_zarr_count.shape[0] / orginal_zarr_count.chunks[0])

time: 276 μs (started: 2024-08-30 01:49:42 +05:30)


In [15]:
for chunk in range(chunks):
    org_data = orginal_zarr_count.get_block_selection(chunk)
    new_data = new_zarr_count.get_block_selection(chunk)
    # sums check
    assert np.sum(org_data) == np.sum(new_data)
    # shape check
    assert org_data.shape == new_data.shape
    # chunk check
    assert np.array_equal(org_data, new_data)
    print(f'All checks passed! for chunk {chunk}')

All checks passed! for chunk 0
All checks passed! for chunk 1
time: 793 ms (started: 2024-08-30 01:49:43 +05:30)


In [16]:
org_data[...]

array([[     0,      0,      0, ...,      0,      0,      0],
       [172867,      0,      0, ...,      0,      0,      0],
       [     0,      0,      0, ...,      0,      0,      0],
       ...,
       [311225,      0,      0, ...,      0,      0,      0],
       [     0, 474664,      0, ...,      0,      0,      0],
       [  5788,      0,      0, ...,      0,      0,      0]],
      dtype=uint32)

time: 3.27 ms (started: 2024-08-30 01:49:44 +05:30)


In [17]:
new_data[...]

array([[     0,      0,      0, ...,      0,      0,      0],
       [172867,      0,      0, ...,      0,      0,      0],
       [     0,      0,      0, ...,      0,      0,      0],
       ...,
       [311225,      0,      0, ...,      0,      0,      0],
       [     0, 474664,      0, ...,      0,      0,      0],
       [  5788,      0,      0, ...,      0,      0,      0]],
      dtype=uint32)

time: 4.58 ms (started: 2024-08-30 01:49:44 +05:30)


# 10X Genomics Dataset
### Compare Filtering in HDF5 and MEX formats

# 1 Modality

In [18]:
h5_path = './test_data/10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.h5'
mex_path = './test_data/10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix/raw_feature_bc_matrix'

time: 464 μs (started: 2024-08-30 01:49:44 +05:30)


In [19]:
h5_reader = scarf.CrH5Reader(h5_path, is_filtered=False)
h5_writer = scarf.CrToZarr(h5_reader, '../../tmp/h5_10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr')

Filtering out background barcodes:   0%|                                                                      …

time: 6.79 s (started: 2024-08-30 01:49:44 +05:30)


In [20]:
h5_writer.dump()

  0%|                                                                                                         …

time: 4.83 s (started: 2024-08-30 01:49:51 +05:30)


In [21]:
mex_reader = scarf.CrDirReader(mex_path, is_filtered=False)
mex_writer = scarf.CrToZarr(mex_reader, '../../tmp/mex_10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr')

Filtering out background barcodes:   0%|                                                                      …

time: 25.1 s (started: 2024-08-30 01:49:56 +05:30)


In [22]:
mex_writer.dump()

  0%|                                                                                                         …

time: 14.7 s (started: 2024-08-30 01:50:21 +05:30)


In [23]:
# Check
h5_zarr = zarr.open('../../tmp/h5_10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr')
h5_zarr_count = h5_zarr['RNA/counts']
mex_zarr = zarr.open('../../tmp/mex_10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr')
mex_zarr_count = mex_zarr['RNA/counts']

time: 6.84 ms (started: 2024-08-30 01:50:35 +05:30)


In [24]:
print("nChunks check")
assert h5_zarr_count.nchunks == mex_zarr_count.nchunks
print("Shape check")
assert h5_zarr_count.shape == mex_zarr_count.shape
print("Chunks check")
assert h5_zarr_count.chunks == mex_zarr_count.chunks

nChunks check
Shape check
Chunks check
time: 1.34 ms (started: 2024-08-30 01:50:36 +05:30)


In [25]:
chunks = math.ceil(h5_zarr_count.shape[0] / h5_zarr_count.chunks[0])

time: 658 μs (started: 2024-08-30 01:50:36 +05:30)


In [26]:
for chunk in range(chunks):
    h5_data = h5_zarr_count.get_block_selection(chunk)
    mex_data = mex_zarr_count.get_block_selection(chunk)
    # sums check
    assert np.sum(h5_data) == np.sum(mex_data)
    # shape check
    assert h5_data.shape == mex_data.shape
    # chunk check
    assert np.array_equal(h5_data, mex_data)
    print(f'All checks passed! for chunk {chunk}')

All checks passed! for chunk 0
All checks passed! for chunk 1
All checks passed! for chunk 2
All checks passed! for chunk 3
All checks passed! for chunk 4
All checks passed! for chunk 5
All checks passed! for chunk 6
All checks passed! for chunk 7
All checks passed! for chunk 8
All checks passed! for chunk 9
All checks passed! for chunk 10
time: 3.51 s (started: 2024-08-30 01:50:36 +05:30)


In [27]:
h5_data[...]

array([[1, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

time: 6.57 ms (started: 2024-08-30 01:50:39 +05:30)


In [28]:
mex_data[...]

array([[1, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

time: 5.18 ms (started: 2024-08-30 01:50:39 +05:30)


# 2 Modalities

In [29]:
h5_path = './test_data/10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.h5'
mex_path = './test_data/10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix/raw_feature_bc_matrix'

time: 626 μs (started: 2024-08-30 01:50:39 +05:30)


In [30]:
h5_reader = scarf.CrH5Reader(h5_path, is_filtered=False)
h5_writer = scarf.CrToZarr(h5_reader, '../../tmp/h5_10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr')
h5_writer.dump()

Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

time: 21.9 s (started: 2024-08-30 01:50:39 +05:30)


In [31]:
mex_reader = scarf.CrDirReader(mex_path, is_filtered=False)
mex_writer = scarf.CrToZarr(mex_reader, '../../tmp/mex_10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr')
mex_writer.dump()

Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

time: 1min 18s (started: 2024-08-30 01:51:01 +05:30)


In [32]:
# Check
h5_zarr = zarr.open('../../tmp/h5_10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr')
h5_zarr_count = h5_zarr['RNA/counts']
mex_zarr = zarr.open('../../tmp/mex_10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr')
mex_zarr_count = mex_zarr['RNA/counts']

time: 7.78 ms (started: 2024-08-30 01:52:20 +05:30)


In [33]:
print("nChunks check")
assert h5_zarr_count.nchunks == mex_zarr_count.nchunks
print("Shape check")
assert h5_zarr_count.shape == mex_zarr_count.shape
print("Chunks check")
assert h5_zarr_count.chunks == mex_zarr_count.chunks

nChunks check
Shape check
Chunks check
time: 1.16 ms (started: 2024-08-30 01:52:20 +05:30)


In [34]:
nchunks = math.ceil(h5_zarr_count.shape[0] / h5_zarr_count.chunks[0])

time: 804 μs (started: 2024-08-30 01:52:20 +05:30)


In [35]:
for chunk in range(nchunks):
    h5_data = h5_zarr_count.get_block_selection(chunk)
    mex_data = mex_zarr_count.get_block_selection(chunk)
    # sums check
    assert np.sum(h5_data) == np.sum(mex_data)
    # shape check
    assert h5_data.shape == mex_data.shape
    # chunk check
    assert np.array_equal(h5_data, mex_data)
    print(f'All checks passed! for chunk {chunk}')

All checks passed! for chunk 0
All checks passed! for chunk 1
All checks passed! for chunk 2
All checks passed! for chunk 3
All checks passed! for chunk 4
All checks passed! for chunk 5
All checks passed! for chunk 6
All checks passed! for chunk 7
All checks passed! for chunk 8
All checks passed! for chunk 9
All checks passed! for chunk 10
time: 5.31 s (started: 2024-08-30 01:52:21 +05:30)


# Different Filtering Cutoff

In [36]:
def make_test(h5_path, mex_path, h5_save_path, mex_save_path, filter_cut):
    print("Processing HDF5")
    h5_reader = scarf.CrH5Reader(h5_path, is_filtered=False, filtering_cutoff=filter_cut)
    h5_writer = scarf.CrToZarr(h5_reader, h5_save_path)
    h5_writer.dump()

    print("Processing MEX")
    mex_reader = scarf.CrDirReader(mex_path, is_filtered=False, filtering_cutoff=filter_cut)
    mex_writer = scarf.CrToZarr(mex_reader, mex_save_path)
    mex_writer.dump()

    h5_zarr = zarr.open(h5_save_path)
    h5_zarr_count = h5_zarr['RNA/counts']
    mex_zarr = zarr.open(mex_save_path)
    mex_zarr_count = mex_zarr['RNA/counts']

    print("nChunks check")
    assert h5_zarr_count.nchunks == mex_zarr_count.nchunks
    print("Shape check")
    assert h5_zarr_count.shape == mex_zarr_count.shape
    print("Chunks check")
    assert h5_zarr_count.chunks == mex_zarr_count.chunks

    chunks = math.ceil(h5_zarr_count.shape[0] / h5_zarr_count.chunks[0])
    for chunk in range(chunks):
        h5_data = h5_zarr_count.get_block_selection(chunk)
        mex_data = mex_zarr_count.get_block_selection(chunk)
        # sums check
        assert np.sum(h5_data) == np.sum(mex_data)
        # shape check
        assert h5_data.shape == mex_data.shape
        # chunk check
        assert np.array_equal(h5_data, mex_data)
        print(f'All checks passed! for chunk {chunk}')

time: 2.3 ms (started: 2024-08-30 01:52:26 +05:30)


In [37]:
h5_path = './test_data/10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.h5'
mex_path = './test_data/10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix/raw_feature_bc_matrix'

h5_save_path = '../../tmp/h5_100_10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr'
mex_save_path = '../../tmp/mex_100_10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr'

make_test(h5_path, mex_path, h5_save_path, mex_save_path, 100)

Processing HDF5


Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

Processing MEX


Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

nChunks check
Shape check
Chunks check
All checks passed! for chunk 0
All checks passed! for chunk 1
All checks passed! for chunk 2
All checks passed! for chunk 3
All checks passed! for chunk 4
All checks passed! for chunk 5
All checks passed! for chunk 6
All checks passed! for chunk 7
All checks passed! for chunk 8
All checks passed! for chunk 9
All checks passed! for chunk 10
All checks passed! for chunk 11
All checks passed! for chunk 12
All checks passed! for chunk 13
All checks passed! for chunk 14
All checks passed! for chunk 15
All checks passed! for chunk 16
All checks passed! for chunk 17
All checks passed! for chunk 18
All checks passed! for chunk 19
All checks passed! for chunk 20
All checks passed! for chunk 21
All checks passed! for chunk 22
All checks passed! for chunk 23
All checks passed! for chunk 24
All checks passed! for chunk 25
All checks passed! for chunk 26
All checks passed! for chunk 27
All checks passed! for chunk 28
All checks passed! for chunk 29
All checks 

In [38]:
h5_path = './test_data/10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.h5'
mex_path = './test_data/10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix/raw_feature_bc_matrix'

h5_save_path = '../../tmp/h5_10000_10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr'
mex_save_path = '../../tmp/mex_10000_10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr'

make_test(h5_path, mex_path, h5_save_path, mex_save_path, 10000)

Processing HDF5


Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

Processing MEX


Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

nChunks check
Shape check
Chunks check
All checks passed! for chunk 0
All checks passed! for chunk 1
All checks passed! for chunk 2
time: 42.8 s (started: 2024-08-30 01:53:38 +05:30)


Multimodal

In [39]:
h5_path = './test_data/10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.h5'
mex_path = './test_data/10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix/raw_feature_bc_matrix'

h5_save_path = '../../tmp/h5_100_10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr'
mex_save_path = '../../tmp/mex_100_10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr'

make_test(h5_path, mex_path, h5_save_path, mex_save_path, 100)

Processing HDF5


Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

Processing MEX


Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

nChunks check
Shape check
Chunks check
All checks passed! for chunk 0
All checks passed! for chunk 1
All checks passed! for chunk 2
All checks passed! for chunk 3
All checks passed! for chunk 4
All checks passed! for chunk 5
All checks passed! for chunk 6
All checks passed! for chunk 7
All checks passed! for chunk 8
All checks passed! for chunk 9
All checks passed! for chunk 10
All checks passed! for chunk 11
All checks passed! for chunk 12
All checks passed! for chunk 13
All checks passed! for chunk 14
All checks passed! for chunk 15
All checks passed! for chunk 16
All checks passed! for chunk 17
All checks passed! for chunk 18
All checks passed! for chunk 19
All checks passed! for chunk 20
All checks passed! for chunk 21
All checks passed! for chunk 22
All checks passed! for chunk 23
All checks passed! for chunk 24
All checks passed! for chunk 25
All checks passed! for chunk 26
time: 1min 31s (started: 2024-08-30 01:54:20 +05:30)


In [40]:
h5_path = './test_data/10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.h5'
mex_path = './test_data/10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix/raw_feature_bc_matrix'

h5_save_path = '../../tmp/h5_10000_10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr'
mex_save_path = '../../tmp/mex_10000_10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr'

make_test(h5_path, mex_path, h5_save_path, mex_save_path, 10000)

Processing HDF5


Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

Processing MEX


Filtering out background barcodes:   0%|                                                                      …

  0%|                                                                                                         …

nChunks check
Shape check
Chunks check
All checks passed! for chunk 0
All checks passed! for chunk 1
All checks passed! for chunk 2
All checks passed! for chunk 3
All checks passed! for chunk 4
All checks passed! for chunk 5
All checks passed! for chunk 6
All checks passed! for chunk 7
All checks passed! for chunk 8
All checks passed! for chunk 9
time: 1min 30s (started: 2024-08-30 01:55:52 +05:30)
